In [121]:
import pickle

import numpy as np
import pandas as pd

from statsmodels.stats.multitest import multipletests

In [122]:
with open('data/emotion_testing_dct.pkl', mode='rb') as f:
    emotion_testing_dct = pickle.load(f)
with open('data/speech_vocal_testing_dct.pkl', mode='rb') as f:
    speech_vocal_testing_dct = pickle.load(f)

In [123]:
from collections import defaultdict

In [124]:
from scipy.stats import ttest_ind

In [125]:
overall_testing_dct = defaultdict(lambda: [])
for key, value in emotion_testing_dct.items():
    for i in range(4):
        _, pval = ttest_ind(
            value[f'judge_{i+1}']['winners'], value[f'judge_{i+1}']['losers'], equal_var=False
        )
        overall_testing_dct[key].append(pval)
        
for key, value in speech_vocal_testing_dct.items():
    for i in range(4):
        _, pval = ttest_ind(
            value[f'judge_{i+1}']['winners'], value[f'judge_{i+1}']['losers'], equal_var=False
        )
        overall_testing_dct[key].append(pval)

In [126]:
dict(overall_testing_dct)

{'neutral_positive': [0.9958899945337658,
  0.20312552570002246,
  0.6661290505129572,
  0.7884670569228887],
 'instability': [0.9323061898150982,
  0.09179322092296746,
  0.01946391503359387,
  0.6958646465221325],
 'speech_rate': [0.15163732646425,
  0.8340294518216509,
  0.6144514554066205,
  0.4731137870582077],
 'speech_dur': [0.17191096778179396,
  0.8724518591404847,
  0.0024713612788371682,
  0.4839871410427736],
 'speech_interrupt': [0.9009157159160461,
  0.052194845268007925,
  0.5656832477106595,
  0.1545963735987405],
 'speech_question': [0.15210326903655805,
  0.43775861532894667,
  0.08926125647672874,
  0.017141715773884725],
 'speech_situation': [0.5630158497200421,
  0.3895427068487255,
  0.986777269019279,
  0.24660257781994788],
 'speech_role': [0.9572690369520962,
  0.34619330074283927,
  0.3994673866854833,
  0.6298334140112364],
 'speech_role_opponent': [0.44537584430776134,
  0.5714525669475476,
  0.5603265599229401,
  0.06888017018780528],
 'speech_relevance': [

In [127]:
result = multipletests(
    pvals=[x for value in overall_testing_dct.values() for x in value],
    alpha=0.10, method='fdr_bh'
)

In [128]:
result

(array([False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False]),
 array([0.99588999, 0.81192301, 0.90274333, 0.94616047, 0.99445994,
        0.62943923, 0.31142264, 0.90274333, 0.74206259, 0.97389975,
        0.90274333, 0.86042158, 0.75015695, 0.97389975, 0.11862534,
        0.86042158, 0.98281714, 0.62633814, 0.88482978, 0.74206259,
        0.74206259, 0.86042158, 0.62943923, 0.31142264, 0.88482978,
        0.86042158, 0.99588999, 0.84549455, 0.99588999, 0.86042158,
        0.86042158, 0.90274333, 0.86042158, 0.88482978, 0.88482978,
        0.62943923, 0.86042158, 0.92287594, 0.86042158, 0.86042158,
        0.90274333, 0.97389975, 0.86042158, 0.86042158, 0.90274333,
  

### Logit

In [129]:
import statsmodels.api as sm
from functools import reduce

In [130]:
big_dct = {}

for key, value in speech_vocal_testing_dct.items():
    big_dct[key] = value

for key, value in emotion_testing_dct.items():
    big_dct[key] = value

In [131]:
all_titles = [value['titles'] for value in big_dct.values()]
all_unique_titles = list(
    reduce(lambda a, b: set(a).intersection(set(b)), all_titles)
)

In [132]:
from copy import deepcopy

models = {'judge_1': None, 'judge_2': None, 'judge_3': None, 'judge_4': None}
filtered_models = deepcopy(models)

for judge_n in ['judge_1', 'judge_2', 'judge_3', 'judge_4']:
    win_dataset_dct = defaultdict(lambda: [])
    lose_dataset_dct = defaultdict(lambda: [])

    for title in all_unique_titles:
        for key, value in big_dct.items():
            proper_idx = value['titles'].index(title)
            win_dataset_dct[key].append(value[judge_n]['winners'][proper_idx])
            lose_dataset_dct[key].append(value[judge_n]['losers'][proper_idx])

    data = pd.DataFrame(win_dataset_dct)
    data = pd.concat([data, pd.DataFrame(lose_dataset_dct)], axis=0, ignore_index=True)

    X = sm.add_constant(data)
    y = [1]*23 + [0]*23
    
    model = sm.Logit(y, X)
    fitted_model = model.fit()
    models[judge_n] = deepcopy(fitted_model)
    
    var_pvals = fitted_model.pvalues
    sig_vars = var_pvals[var_pvals < 0.1].index.tolist()
    
    new_X = sm.add_constant(X[sig_vars])
    filtered_model = sm.Logit(y, new_X)
    fitted_filtered_model = filtered_model.fit()
    
    filtered_models[judge_n] = deepcopy(fitted_filtered_model)

Optimization terminated successfully.
         Current function value: 0.578714
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.615696
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.566967
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.651324
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.479302
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.550173
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.461850
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.577037
         Iterations 6


In [133]:
for i in range(4):
    print(f'judge_{i+1}')
    print(filtered_models[f'judge_{i+1}'].summary())
    print(np.exp(filtered_models[f'judge_{i+1}'].params))
    print()

judge_1
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                   46
Model:                          Logit   Df Residuals:                       43
Method:                           MLE   Df Model:                            2
Date:                Sun, 19 Jun 2022   Pseudo R-squ.:                  0.1117
Time:                        12:58:46   Log-Likelihood:                -28.322
converged:                       True   LL-Null:                       -31.885
Covariance Type:            nonrobust   LLR p-value:                   0.02836
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -8.4340      3.498     -2.411      0.016     -15.290      -1.578
speech_rate     1.9233      1.098      1.752      0.080      -0.228       4.075
speech_dur      0.0194      0.010      1

### Ordered model